In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import fpl_function as fpl
pd.options.mode.chained_assignment = None
sns.set_theme()

In [2]:
GAMEWEEKS_PLAYED = 3

In [3]:
df = pd.read_excel('players_Gameweek_{}.xlsx'.format(GAMEWEEKS_PLAYED))
fixtures = pd.read_excel('fixtures.xlsx')

In [4]:
df.head()

,event_points,now_cost,points_per_game,selected_by_percent,status,team,total_points,web_name,minutes,goals_scored,...,penalties_order,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,starts_per_90,clean_sheets_per_90,position
0,0,54,0.0,0.0,u,Arsenal,0,Fábio Vieira,0,0,...,4.0,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,Midfielder
1,0,68,0.0,0.9,i,Arsenal,0,G.Jesus,5,0,...,NaN,0.00,0.0,0.00,0.00,2.70,0.00,0.0,0.00,Forward
2,2,60,4.7,12.7,a,Arsenal,14,Gabriel,270,0,...,NaN,0.08,0.0,0.02,0.10,1.19,0.33,1.0,0.67,Defender
3,8,81,7.3,18.0,a,Arsenal,22,Havertz,270,2,...,3.0,0.48,0.0,0.04,0.52,1.19,0.33,1.0,0.67,Forward
4,0,40,0.0,0.1,u,Arsenal,0,Hein,0,0,...,NaN,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,Goalkeeper


In [5]:
df.drop(df[df['status'] == 'u'].index, inplace=True)

df['now_cost'] = df['now_cost']/10

#create points per million (cost) column (ppm)
df['ppm'] = round(df['total_points']/df['now_cost'], 2)

#create expected goal involvements vs actual goal involvements column
df['gi_vs_xgi'] = df['goals_scored'] + df['assists'] - df['expected_goal_involvements']

df['goal_involvements'] = df['goals_scored'] + df['assists']

df['points_per_minute'] = df['total_points']/df['minutes']

df['gc_vs_xgc'] = df['goals_conceded'] - df['expected_goals_conceded']

df['percent_of_season_played'] = np.round(df['minutes']/(GAMEWEEKS_PLAYED*90) * 100, 2)

In [6]:
fixtures_going_forward = fixtures.loc[fixtures['Gameweek'] > GAMEWEEKS_PLAYED]

df['next_match'] = df['team'].apply(lambda x: list(fpl.fixtures_by_team(fixtures, x, GAMEWEEKS_PLAYED+1)[['opponent', 'h_or_a']].iloc[-1]))
df['next_5_FDRs'] = df['team'].apply(lambda x: list(fpl.fixtures_by_team(fixtures_going_forward, x, GAMEWEEKS_PLAYED+5)['FDR']))
df['next_5_avg_FDRs'] = df['next_5_FDRs'].apply(lambda x: np.mean(x))

In [7]:
fpl.top_players(df, 'Goalkeeper', 'expected_goals_conceded', False, 10)

,web_name,team,now_cost,selected_by_percent,bps,minutes,percent_of_season_played,ppm,next_match,next_5_avg_FDRs,clean_sheets,saves,goals_conceded,expected_goals_conceded,gc_vs_xgc,total_points
259,Pickford,Everton,4.9,11.1,2,270,100.00,0.82,"[Aston Villa, Away]",2.6,0,9,10,6.08,3.92,4
99,Flekken,Brentford,4.5,11.1,56,270,100.00,2.22,"[Man City, Away]",3.2,0,17,4,5.39,-1.39,10
473,Pope,Newcastle,5.0,8.8,57,270,100.00,3.20,"[Wolves, Away]",3.0,1,12,2,5.26,-3.26,16
219,Henderson,Crystal Palace,4.5,14.7,24,270,100.00,1.56,"[Leicester, Home]",2.8,0,10,5,5.25,-0.25,7
593,Areola,West Ham,4.5,8.5,31,225,83.33,1.78,"[Fulham, Away]",2.8,1,6,4,5.09,-1.09,8
343,Hermansen,Leicester,4.5,0.5,36,270,100.00,1.56,"[Crystal Palace, Away]",2.8,0,12,5,4.56,0.44,7
325,Muric,Ipswich,4.4,0.4,0,180,66.67,0.68,"[Brighton, Away]",2.2,0,4,5,4.29,0.71,3
440,Onana,Man Utd,5.0,5.3,38,270,100.00,1.60,"[Southampton, Away]",2.8,1,4,5,4.26,0.74,8
47,Martinez,Aston Villa,5.0,13.1,33,270,100.00,1.20,"[Everton, Home]",2.4,0,7,4,3.78,0.22,6
584,Vicario,Spurs,5.0,3.9,50,270,100.00,1.80,"[Arsenal, Home]",2.8,1,4,3,3.59,-0.59,9


In [8]:
fpl.top_players(df, 'Goalkeeper', 'gc_vs_xgc', False, 10)

,web_name,team,now_cost,selected_by_percent,bps,minutes,percent_of_season_played,ppm,next_match,next_5_avg_FDRs,clean_sheets,saves,goals_conceded,expected_goals_conceded,gc_vs_xgc,total_points
639,José Sá,Wolves,4.5,2.1,-1,180,66.67,0.22,"[Newcastle, Home]",3.6,0,6,8,2.80,5.20,1
259,Pickford,Everton,4.9,11.1,2,270,100.00,0.82,"[Aston Villa, Away]",2.6,0,9,10,6.08,3.92,4
201,Sánchez,Chelsea,4.5,4.1,31,270,100.00,1.11,"[Bournemouth, Away]",2.6,0,7,5,3.13,1.87,5
440,Onana,Man Utd,5.0,5.3,38,270,100.00,1.60,"[Southampton, Away]",2.8,1,4,5,4.26,0.74,8
325,Muric,Ipswich,4.4,0.4,0,180,66.67,0.68,"[Brighton, Away]",2.2,0,4,5,4.29,0.71,3
343,Hermansen,Leicester,4.5,0.5,36,270,100.00,1.56,"[Crystal Palace, Away]",2.8,0,12,5,4.56,0.44,7
517,Ramsdale,Southampton,4.4,1.5,5,90,33.33,0.45,"[Man Utd, Home]",2.8,0,4,3,2.78,0.22,2
47,Martinez,Aston Villa,5.0,13.1,33,270,100.00,1.20,"[Everton, Home]",2.4,0,7,4,3.78,0.22,6
89,Arrizabalaga,Bournemouth,4.5,0.5,12,90,33.33,0.67,"[Chelsea, Home]",3.4,0,6,2,1.97,0.03,3
379,Kelleher,Liverpool,4.4,0.3,0,0,0.00,0.00,"[Nott'm Forest, Home]",2.4,0,0,0,0.00,0.00,0


In [9]:
fpl.top_players(df, 'Defender', 'expected_goal_involvements', False, 10)

,web_name,team,now_cost,selected_by_percent,bps,minutes,percent_of_season_played,ppm,next_match,next_5_avg_FDRs,...,assists,goal_involvements,expected_goal_involvements,gi_vs_xgi,expected_goal_involvements_per_90,clean_sheets,goals_conceded,expected_goals_conceded,gc_vs_xgc,total_points
366,Alexander-Arnold,Liverpool,7.0,30.2,77,222,82.22,2.86,"[Nott'm Forest, Home]",2.4,...,0,0,1.70,-1.70,0.68,3,0,1.60,-1.60,20
513,Milenković,Nott'm Forest,4.5,0.2,26,180,66.67,1.78,"[Liverpool, Away]",3.0,...,0,0,1.00,-1.00,0.50,1,1,0.88,0.12,8
575,Romero,Spurs,5.1,9.6,74,270,100.00,3.33,"[Arsenal, Home]",2.8,...,0,1,0.97,0.03,0.33,1,3,3.59,-0.59,17
572,Pedro Porro,Spurs,5.5,35.3,55,270,100.00,2.91,"[Arsenal, Home]",2.8,...,0,1,0.93,0.07,0.31,1,3,3.53,-0.53,16
545,Sugawara,Southampton,4.5,0.2,40,206,76.30,2.22,"[Man Utd, Home]",2.8,...,0,1,0.82,0.18,0.36,0,5,5.17,-0.17,10
409,Lewis,Man City,4.6,7.4,55,270,100.00,3.04,"[Brentford, Home]",3.0,...,1,1,0.78,0.22,0.26,1,2,2.03,-0.03,14
389,Robertson,Liverpool,6.0,5.9,78,250,92.59,3.00,"[Nott'm Forest, Home]",2.4,...,0,0,0.69,-0.69,0.25,3,0,2.11,-2.11,18
404,Gvardiol,Man City,6.0,29.9,59,267,98.89,1.83,"[Brentford, Home]",3.0,...,0,0,0.64,-0.64,0.21,1,2,1.94,0.06,11
246,Coleman,Everton,4.4,0.0,-1,90,33.33,0.23,"[Aston Villa, Away]",2.6,...,0,0,0.63,-0.63,0.63,0,3,2.22,0.78,1
618,Aït-Nouri,Wolves,4.5,2.5,0,209,77.41,0.89,"[Newcastle, Home]",3.6,...,1,1,0.62,0.38,0.27,0,8,2.84,5.16,4


In [10]:
fpl.top_players(df, 'Midfielder', 'expected_goal_involvements', False, 10)

,web_name,team,now_cost,selected_by_percent,bps,minutes,percent_of_season_played,ppm,next_match,next_5_avg_FDRs,goals_scored,assists,goal_involvements,expected_goal_involvements,gi_vs_xgi,expected_goal_involvements_per_90,total_points
399,De Bruyne,Man City,9.6,11.1,82,266,98.52,1.67,"[Brentford, Home]",3.0,1,1,2,2.76,-0.76,0.93,16
382,M.Salah,Liverpool,12.6,40.4,132,262,97.04,3.25,"[Nott'm Forest, Home]",2.4,3,3,6,2.73,3.27,0.93,41
82,Semenyo,Bournemouth,5.5,4.5,68,270,100.00,3.45,"[Chelsea, Home]",3.4,2,1,3,2.51,0.49,0.83,19
381,Luis Díaz,Liverpool,7.6,14.0,109,226,83.70,4.21,"[Nott'm Forest, Home]",2.4,3,1,4,2.49,1.51,0.99,32
86,Tavernier,Bournemouth,5.5,0.3,52,264,97.78,2.36,"[Chelsea, Home]",3.4,1,0,1,2.30,-1.30,0.78,13
423,B.Fernandes,Man Utd,8.4,12.5,41,258,95.56,0.83,"[Southampton, Away]",2.8,0,0,0,2.11,-2.11,0.74,7
13,Saka,Arsenal,10.1,33.0,84,256,94.81,2.38,"[Spurs, Away]",3.0,1,3,4,2.02,1.98,0.71,24
571,Maddison,Spurs,7.5,4.7,63,237,87.78,1.60,"[Arsenal, Home]",2.8,0,2,2,1.93,0.07,0.73,12
198,Palmer,Chelsea,10.6,42.0,108,262,97.04,2.36,"[Bournemouth, Away]",2.6,1,4,5,1.89,3.11,0.65,25
107,Mbeumo,Brentford,7.1,7.5,105,269,99.63,3.38,"[Man City, Away]",3.2,3,0,3,1.84,1.16,0.61,24


In [11]:
fpl.top_players(df, 'Forward', 'expected_goal_involvements', False, 10)

,web_name,team,now_cost,selected_by_percent,bps,minutes,percent_of_season_played,ppm,next_match,next_5_avg_FDRs,goals_scored,assists,goal_involvements,expected_goal_involvements,gi_vs_xgi,expected_goal_involvements_per_90,total_points
405,Haaland,Man City,15.2,65.0,204,269,99.63,2.70,"[Brentford, Home]",3.0,7,0,7,3.93,3.07,1.31,41
118,Wissa,Brentford,6.1,7.6,90,245,90.74,3.44,"[Man City, Away]",3.2,2,1,3,2.36,0.64,0.87,21
196,N.Jackson,Chelsea,7.6,5.7,69,222,82.22,2.37,"[Bournemouth, Away]",2.6,2,1,3,2.05,0.95,0.83,18
626,Cunha,Wolves,6.5,2.6,58,190,70.37,1.38,"[Newcastle, Home]",3.6,1,0,1,1.79,-0.79,0.85,9
58,Watkins,Aston Villa,8.9,26.5,14,185,68.52,1.01,"[Everton, Home]",2.4,0,1,1,1.59,-0.59,0.77,9
3,Havertz,Arsenal,8.1,18.0,80,270,100.00,2.72,"[Spurs, Away]",3.0,2,1,3,1.55,1.45,0.52,22
509,Wood,Nott'm Forest,6.1,7.9,66,235,87.04,3.11,"[Liverpool, Away]",3.0,2,0,2,1.54,0.46,0.59,19
160,Welbeck,Brighton,5.7,13.7,82,252,93.33,4.04,"[Ipswich, Home]",3.0,2,1,3,1.48,1.52,0.52,23
461,Isak,Newcastle,8.5,44.7,62,270,100.00,1.88,"[Wolves, Away]",3.0,1,1,2,1.38,0.62,0.46,16
141,João Pedro,Brighton,5.7,26.5,84,261,96.67,3.16,"[Ipswich, Home]",3.0,2,0,2,1.14,0.86,0.39,18


In [12]:
fpl.top_players(df, 'Forward', 'ppm', False, 10)

,web_name,team,now_cost,selected_by_percent,bps,minutes,percent_of_season_played,ppm,next_match,next_5_avg_FDRs,goals_scored,assists,goal_involvements,expected_goal_involvements,gi_vs_xgi,expected_goal_involvements_per_90,total_points
160,Welbeck,Brighton,5.7,13.7,82,252,93.33,4.04,"[Ipswich, Home]",3.0,2,1,3,1.48,1.52,0.52,23
118,Wissa,Brentford,6.1,7.6,90,245,90.74,3.44,"[Man City, Away]",3.2,2,1,3,2.36,0.64,0.87,21
141,João Pedro,Brighton,5.7,26.5,84,261,96.67,3.16,"[Ipswich, Home]",3.0,2,0,2,1.14,0.86,0.39,18
509,Wood,Nott'm Forest,6.1,7.9,66,235,87.04,3.11,"[Liverpool, Away]",3.0,2,0,2,1.54,0.46,0.59,19
3,Havertz,Arsenal,8.1,18.0,80,270,100.00,2.72,"[Spurs, Away]",3.0,2,1,3,1.55,1.45,0.52,22
245,Calvert-Lewin,Everton,5.9,1.7,46,233,86.30,2.71,"[Aston Villa, Away]",2.6,1,1,2,0.77,1.23,0.30,16
405,Haaland,Man City,15.2,65.0,204,269,99.63,2.70,"[Brentford, Home]",3.0,7,0,7,3.93,3.07,1.31,41
196,N.Jackson,Chelsea,7.6,5.7,69,222,82.22,2.37,"[Bournemouth, Away]",2.6,2,1,3,2.05,0.95,0.83,18
38,Duran,Aston Villa,6.0,1.1,58,82,30.37,2.33,"[Everton, Home]",2.4,2,0,2,0.85,1.15,0.93,14
307,Delap,Ipswich,5.5,0.7,35,235,87.04,2.00,"[Brighton, Away]",2.2,1,0,1,0.22,0.78,0.08,11


In [13]:
defs = fpl.top_players(df, 'Defender', 'expected_goal_involvements', False, 3)
mids = fpl.top_players(df, 'Midfielder', 'expected_goal_involvements', False, 4)
frwd = fpl.top_players(df, 'Forward', 'expected_goal_involvements', False, 3)

three43 = pd.concat([defs, mids, frwd])

In [14]:
three43

,web_name,team,now_cost,selected_by_percent,bps,minutes,percent_of_season_played,ppm,next_match,next_5_avg_FDRs,...,assists,goal_involvements,expected_goal_involvements,gi_vs_xgi,expected_goal_involvements_per_90,clean_sheets,goals_conceded,expected_goals_conceded,gc_vs_xgc,total_points
366,Alexander-Arnold,Liverpool,7.0,30.2,77,222,82.22,2.86,"[Nott'm Forest, Home]",2.4,...,0,0,1.70,-1.70,0.68,3.0,0.0,1.60,-1.60,20
513,Milenković,Nott'm Forest,4.5,0.2,26,180,66.67,1.78,"[Liverpool, Away]",3.0,...,0,0,1.00,-1.00,0.50,1.0,1.0,0.88,0.12,8
575,Romero,Spurs,5.1,9.6,74,270,100.00,3.33,"[Arsenal, Home]",2.8,...,0,1,0.97,0.03,0.33,1.0,3.0,3.59,-0.59,17
399,De Bruyne,Man City,9.6,11.1,82,266,98.52,1.67,"[Brentford, Home]",3.0,...,1,2,2.76,-0.76,0.93,NaN,NaN,NaN,NaN,16
382,M.Salah,Liverpool,12.6,40.4,132,262,97.04,3.25,"[Nott'm Forest, Home]",2.4,...,3,6,2.73,3.27,0.93,NaN,NaN,NaN,NaN,41
82,Semenyo,Bournemouth,5.5,4.5,68,270,100.00,3.45,"[Chelsea, Home]",3.4,...,1,3,2.51,0.49,0.83,NaN,NaN,NaN,NaN,19
381,Luis Díaz,Liverpool,7.6,14.0,109,226,83.70,4.21,"[Nott'm Forest, Home]",2.4,...,1,4,2.49,1.51,0.99,NaN,NaN,NaN,NaN,32
405,Haaland,Man City,15.2,65.0,204,269,99.63,2.70,"[Brentford, Home]",3.0,...,0,7,3.93,3.07,1.31,NaN,NaN,NaN,NaN,41
118,Wissa,Brentford,6.1,7.6,90,245,90.74,3.44,"[Man City, Away]",3.2,...,1,3,2.36,0.64,0.87,NaN,NaN,NaN,NaN,21
196,N.Jackson,Chelsea,7.6,5.7,69,222,82.22,2.37,"[Bournemouth, Away]",2.6,...,1,3,2.05,0.95,0.83,NaN,NaN,NaN,NaN,18


In [15]:
print('Cost: {}; xGI: {}, Total Points: {}'.format(three43['now_cost'].sum(), three43['expected_goal_involvements'].sum(), three43['total_points'].sum()))

Cost: 80.79999999999998; xGI: 22.5, Total Points: 233


In [16]:
fpl.top_players_by_cost(df, 'Forward', 5.7, 'expected_goal_involvements', 10)

,web_name,team,now_cost,selected_by_percent,bps,minutes,percent_of_season_played,ppm,next_match,next_5_avg_FDRs,goals_scored,assists,goal_involvements,expected_goal_involvements,gi_vs_xgi,expected_goal_involvements_per_90,total_points
160,Welbeck,Brighton,5.7,13.7,82,252,93.33,4.04,"[Ipswich, Home]",3.0,2,1,3,1.48,1.52,0.52,23
141,João Pedro,Brighton,5.7,26.5,84,261,96.67,3.16,"[Ipswich, Home]",3.0,2,0,2,1.14,0.86,0.39,18
524,Armstrong,Southampton,5.5,1.4,6,205,75.93,0.73,"[Man Utd, Home]",2.8,0,0,0,0.92,-0.92,0.40,4
650,Strand Larsen,Wolves,5.5,0.9,43,258,95.56,1.82,"[Newcastle, Home]",3.6,1,0,1,0.86,0.14,0.30,10
592,Antonio,West Ham,5.5,1.4,9,203,75.19,1.09,"[Fulham, Away]",2.8,0,0,0,0.83,-0.83,0.36,6
356,Vardy,Leicester,5.6,7.0,37,238,88.15,1.96,"[Crystal Palace, Away]",2.8,1,0,1,0.82,0.18,0.31,11
334,Edouard,Leicester,5.4,0.6,3,115,42.59,0.56,"[Crystal Palace, Away]",2.8,0,0,0,0.42,-0.42,0.33,3
257,Ndiaye,Everton,5.4,0.2,10,142,52.59,0.74,"[Aston Villa, Away]",2.6,0,0,0,0.36,-0.36,0.23,4
601,Ings,West Ham,5.0,1.1,2,5,1.85,0.20,"[Fulham, Away]",2.8,0,0,0,0.25,-0.25,4.50,1
307,Delap,Ipswich,5.5,0.7,35,235,87.04,2.00,"[Brighton, Away]",2.2,1,0,1,0.22,0.78,0.08,11


In [17]:
fpl.top_players_by_cost(df, 'Midfielder', 7.5, 'expected_goal_involvements', 10)

,web_name,team,now_cost,selected_by_percent,bps,minutes,percent_of_season_played,ppm,next_match,next_5_avg_FDRs,goals_scored,assists,goal_involvements,expected_goal_involvements,gi_vs_xgi,expected_goal_involvements_per_90,total_points
82,Semenyo,Bournemouth,5.5,4.5,68,270,100.00,3.45,"[Chelsea, Home]",3.4,2,1,3,2.51,0.49,0.83,19
86,Tavernier,Bournemouth,5.5,0.3,52,264,97.78,2.36,"[Chelsea, Home]",3.4,1,0,1,2.30,-1.30,0.78,13
571,Maddison,Spurs,7.5,4.7,63,237,87.78,1.60,"[Arsenal, Home]",2.8,0,2,2,1.93,0.07,0.73,12
107,Mbeumo,Brentford,7.1,7.5,105,269,99.63,3.38,"[Man City, Away]",3.2,3,0,3,1.84,1.16,0.61,24
148,Mitoma,Brighton,6.6,7.9,60,261,96.67,2.42,"[Ipswich, Home]",3.0,1,1,2,1.77,0.23,0.61,16
608,Souček,West Ham,5.0,3.3,38,180,66.67,2.80,"[Fulham, Away]",2.8,1,1,2,1.68,0.32,0.84,14
429,Garnacho,Man Utd,6.4,12.0,20,122,45.19,1.09,"[Southampton, Away]",2.8,0,1,1,1.64,-0.64,1.21,7
255,McNeil,Everton,5.4,0.6,66,270,100.00,1.85,"[Aston Villa, Away]",2.6,0,1,1,1.63,-0.63,0.55,10
496,Gibbs-White,Nott'm Forest,6.5,4.3,56,270,100.00,2.00,"[Liverpool, Away]",3.0,1,0,1,1.49,-0.49,0.49,13
147,Minteh,Brighton,5.5,1.6,34,205,75.93,1.82,"[Ipswich, Home]",3.0,0,2,2,1.38,0.62,0.61,10
